### Feed-forward Neural Network

An acyclic artificial neural network in which data moves in only one direction, i.e. forward—from the input nodes, through the hidden nodes (if any) and to the output nodes. It is the simplest form of artificial neural networks.

In the example, we use 3-layer neural network with variable number of node in the hidden layer. The purpose of the model is to predict the quality of white wine using the wine dataset.

In [223]:
# Importing libraries
import os
import pickle, gzip
import urllib.request

import struct
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [258]:
# Variable parameters
n_h = 16;
learning_rate = 0.01;
epochs = 5000;
validation_error = 0.01;

In [226]:
# Loading dataset
df = pd.read_csv("winequality-white.csv", sep=';')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [227]:
# Normalizing dataset
def normalize_dataset(ds):
    result = ds.copy()
    for feature_name in ds.columns:
        max_value = ds[feature_name].max()
        min_value = ds[feature_name].min()
        result[feature_name] = (ds[feature_name] - min_value) / (max_value - min_value)
    return result

In [228]:
wine_ds = normalize_dataset(df)
wine_ds.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,0.307692,0.186275,0.216867,0.308282,0.106825,0.149826,0.373550,0.267785,0.254545,0.267442,0.129032,0.5
1,0.240385,0.215686,0.204819,0.015337,0.118694,0.041812,0.285383,0.132832,0.527273,0.313953,0.241935,0.5
2,0.413462,0.196078,0.240964,0.096626,0.121662,0.097561,0.204176,0.154039,0.490909,0.255814,0.338710,0.5
3,0.326923,0.147059,0.192771,0.121166,0.145401,0.156794,0.410673,0.163678,0.427273,0.209302,0.306452,0.5
4,0.326923,0.147059,0.192771,0.121166,0.145401,0.156794,0.410673,0.163678,0.427273,0.209302,0.306452,0.5


In [229]:
X = wine_ds.drop(['quality'], axis=1).values
Y = wine_ds['quality'].values
print(X.shape, Y.shape)

(4898, 11) (4898,)


In [230]:
# Splitting dataset into training and testing set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=20)

In [231]:
# Initializing parameters
def initialize_parameters(n_input, n_hidden, n_output):
    
    np.random.seed(1);
    W1 = np.random.randn(n_h, n_input) * 0.01
    b1 = np.zeros(shape=(n_h, 1))
    W2 = np.random.randn(n_output, n_h) * 0.01
    b2 = np.zeros(shape=(n_output, 1))
    
    parameters = {'W1': W1,
                  'b1': b1,
                  'W2': W2,
                  'b2': b2};
    
    return parameters;

In [232]:
# Creating forward function
def linear_function(X, W, b):
    
#    print(W.shape, ' || ', X.shape, ' || ', b.shape)
    Z = W @ X + b;
    cache = (X, W, b);
    
    return Z, cache;

def sigmoid(Z):

    Z = Z.astype('float64')
    A = 1/(1+np.exp(-Z));
    A_cache = (Z);
    
    return A, A_cache;


In [233]:
# Cost function
def cost_function(A, AL):
    
    J = 0.5 * np.square(AL - A);
    return J;

In [234]:
#  Creating backward function
def backward_function(dZ, cache):
    
    X_prev, W, b = cache
    m = X_prev.shape[0];
    
    dW = 1 / m * np.dot(dZ, X_prev.T);
    db = 1 / m * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T,dZ)
    
    return dA_prev, dW, db;

In [235]:
# updating weights and biases using gradient descent
def gradient_descent(parameters, gradients, lr):
    
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
   
    dW1 = gradients['dW1']
    db1 = gradients['db1']
    dW2 = gradients['dW2']
    db2 = gradients['db2']
    W1 = W1 - lr * dW1
    b1 = b1 - lr * db1
    W2 = W2 - lr * dW2
    b2 = b2 - lr * db2
    
    parameters = {"W1": W1, "b1": b1,"W2": W2,"b2": b2}
    
    return parameters;

In [243]:
def propagate(X, Y, parameters, hidden_layer_size):
    
    Z1, cache1 = linear_function(X.T, parameters['W1'], parameters['b1']);
    A1, A1_cache = sigmoid(Z1);
    Z2, cache2 = linear_function(A1, parameters['W2'], parameters['b2']);
    A2, A2_cache = sigmoid(Z2);
    
    cost = cost_function(A2, Y);
    
    dZ2 = A2 - Y;
    dA2_prev, dW2, db2 = backward_function(dZ2, cache2);
    dZ1 = np.multiply(np.dot(parameters['W2'].T, dZ2), 1 - np.power(A1, 2));
    dA1_prev, dW1, db1 = backward_function(dZ1, cache1);
    
    gradients = {"dW1": dW1,
                  "db1": db1,
                  "dW2": dW2,
                  "db2": db2};
    
    return gradients, cost;

In [253]:
# Training the model to update parameters using the training set
def nn_model(X, Y, hl_size, learning_rate, epochs):
    
    parameters = initialize_parameters(X.shape[1], hl_size, 1);
    
    for i in range(epochs):
        gradients, cost = propagate(X, Y, parameters, hl_size)
        parameters = gradient_descent(parameters, gradients, learning_rate)
        
        if i % 250 == 0:
            print ("Cost after iteration %i: %f" %(i, np.average(cost)))
            
    return parameters

In [259]:
parameters = nn_model(X_train, Y_train, n_h, learning_rate, epochs);

Cost after iteration 0: 0.010971
Cost after iteration 250: 0.008811
Cost after iteration 500: 0.008088
Cost after iteration 750: 0.007854
Cost after iteration 1000: 0.007786
Cost after iteration 1250: 0.007763
Cost after iteration 1500: 0.007754
Cost after iteration 1750: 0.007749
Cost after iteration 2000: 0.007747
Cost after iteration 2250: 0.007744
Cost after iteration 2500: 0.007743
Cost after iteration 2750: 0.007741
Cost after iteration 3000: 0.007740
Cost after iteration 3250: 0.007739
Cost after iteration 3500: 0.007738
Cost after iteration 3750: 0.007736
Cost after iteration 4000: 0.007735
Cost after iteration 4250: 0.007734
Cost after iteration 4500: 0.007733
Cost after iteration 4750: 0.007733


In [260]:
# Testing the model to predict the quality of wine using testing set
def predict(X, Y, parameters):
    Z1, cache1 = linear_function(X.T, parameters['W1'], parameters['b1']);
    A1, A1_cache = sigmoid(Z1);
    Z2, cache2 = linear_function(A1, parameters['W2'], parameters['b2']);
    A2, A2_cache = sigmoid(Z2);
    
    prediction = np.round(A2);
    
    accuracy = np.dot(Y, prediction.T) + np.dot(1 - Y, 1 - prediction.T);
    
    return (accuracy / Y.size) * 100;

In [261]:
accuracy = predict(X_test, Y_test, parameters);
print("Accuracy: {0}".format(accuracy));

Accuracy: [56.41156463]


### Convolutional Neural Network

A class of deep neural networks that is most commonly applied to analyzing visual imagery. It is a regularized versin of multi-layer perceptron that takes advantage of the hierarchical pattern in data and assembles more complex patterns using smaller and simpler patterns.

In the example, we use application of handwritten digit recognition using MNIST dataset.

In [3]:
# Importing libraries
import tensorflow.keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K

In [4]:
# Variable parameters
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

In [5]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [6]:
# Adjusting input shape to match the shape in kernel
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [8]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'training samples')
print(x_test.shape[0], 'testing samples')

# Converting class vectors to binary class matrices
y_train = tensorflow.keras.utils.to_categorical(y_train, num_classes)
y_test = tensorflow.keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 training samples
10000 testing samples


In [9]:
# Building the model
model = Sequential()
model.add(Conv2D(6, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(120, activation='relu'))
model.add(Dense(84, activation='relu'))

model.add(Dense(num_classes, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 6)         60        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 16)        880       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 400)               0         
_________________________________________________________________
dense (Dense)                (None, 120)               48120     
_________________________________________________________________
dense_1 (Dense)              (None, 84)                1

In [10]:
# Adding loss and optimier function in the model
model.compile(loss=tensorflow.keras.losses.categorical_crossentropy,
              optimizer=tensorflow.keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0),
              metrics=['accuracy'])

In [11]:
# Training the model using the training dataset
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 6s 106us/sample - loss: 2.3012 - accuracy: 0.1120 - val_loss: 2.2994 - val_accuracy: 0.1135
Epoch 2/12
60000/60000 [==============================] - 6s 93us/sample - loss: 2.2372 - accuracy: 0.1736 - val_loss: 1.5975 - val_accuracy: 0.4569
Epoch 3/12
60000/60000 [==============================] - 6s 93us/sample - loss: 0.6786 - accuracy: 0.7790 - val_loss: 0.3849 - val_accuracy: 0.8799
Epoch 4/12
60000/60000 [==============================] - 6s 95us/sample - loss: 0.3190 - accuracy: 0.9028 - val_loss: 0.2399 - val_accuracy: 0.9247
Epoch 5/12
60000/60000 [==============================] - 6s 105us/sample - loss: 0.2186 - accuracy: 0.9320 - val_loss: 0.1750 - val_accuracy: 0.9450
Epoch 6/12
60000/60000 [==============================] - 7s 110us/sample - loss: 0.1639 - accuracy: 0.9496 - val_loss: 0.1591 - val_accuracy: 0.9490
Epoch 7/12
60000/60000 [=============================

In [12]:
# Evaluating the model using the testing dataset
score = model.evaluate(x_test, y_test, verbose=0)

In [13]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.07047177767213433
Test accuracy: 0.9762


### Recurrent Neural Network

A class of artificial neural networks where connections between nodes form a directed graph along a temporal sequence. it uses its internal memory to process variable length sequences of input which makes it applicable to tasks such as unsegmented, connected handwriting recognition or speech recognition.

We use RNN, especially LSTM, to generate music notes after training the model using existing notes in the form of MIDI dataset. LSTM is a type of RNN which has feddback connections to process entrie sequence of data instead of just a single data point. Each LSTM cell consist of three gates, viz. input gate, output gate and forget gate, that controls the flow of information in and out of the cell.

In [20]:
# Importing libraries
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord, stream
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization as BatchNorm
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint

In [25]:
# Initializing parameters
epochs = 10
batch_size = 128
generated_note_size = 500

data_path = "rnn"
output_file = 'test_output.mid'

In [22]:
# Loading dataset
def get_notes(data_path):
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []

    for file in glob.glob(data_path + "/midi_songs/*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open(data_path + '/data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

In [27]:
notes = get_notes(data_path)

n_vocab = len(set(notes))

Parsing rnn/midi_songs\0fithos.mid
Parsing rnn/midi_songs\8.mid
Parsing rnn/midi_songs\ahead_on_our_way_piano.mid
Parsing rnn/midi_songs\AT.mid
Parsing rnn/midi_songs\balamb.mid
Parsing rnn/midi_songs\bcm.mid
Parsing rnn/midi_songs\BlueStone_LastDungeon.mid
Parsing rnn/midi_songs\braska.mid
Parsing rnn/midi_songs\caitsith.mid
Parsing rnn/midi_songs\Cids.mid
Parsing rnn/midi_songs\cosmo.mid
Parsing rnn/midi_songs\costadsol.mid
Parsing rnn/midi_songs\dayafter.mid
Parsing rnn/midi_songs\decisive.mid
Parsing rnn/midi_songs\dontbeafraid.mid
Parsing rnn/midi_songs\DOS.mid
Parsing rnn/midi_songs\electric_de_chocobo.mid
Parsing rnn/midi_songs\Eternal_Harvest.mid
Parsing rnn/midi_songs\EyesOnMePiano.mid
Parsing rnn/midi_songs\ff11_awakening_piano.mid
Parsing rnn/midi_songs\ff1battp.mid
Parsing rnn/midi_songs\FF3_Battle_(Piano).mid
Parsing rnn/midi_songs\FF3_Third_Phase_Final_(Piano).mid
Parsing rnn/midi_songs\ff4-airship.mid
Parsing rnn/midi_songs\Ff4-BattleLust.mid
Parsing rnn/midi_songs\ff4-f

In [28]:
# Preparing dataset for the model
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = to_categorical(network_output)

    return (network_input, network_output)

In [29]:
network_input, network_output = prepare_sequences(notes, n_vocab)

In [30]:
# Create the structure of the model
model = Sequential()
model.add(LSTM(
    512,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    recurrent_dropout=0.3,
    return_sequences=True
))
model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
model.add(LSTM(512))
model.add(BatchNorm())
model.add(Dropout(0.3))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(BatchNorm())
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 512)          1052672   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
activation (Activation)      (None, 256)              

In [ ]:
# Training the model
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)
callbacks_list = [checkpoint]

history = model.fit(network_input, network_output, epochs=epochs, batch_size=batch_size, callbacks=callbacks_list)

Train on 43990 samples
14080/43990 [========>.....................] - ETA: 1:12:56 - loss: 4.8411

In [1]:
# Generating notes from the neural network based on a sequence of notes
def generate_notes(model, network_input, pitchnames, n_vocab):
    
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    for note_index in range(generated_note_size):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern = numpy.append(pattern, index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [ ]:
pitchnames = sorted(set(item for item in notes))
prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)

In [ ]:
# Converting the output from the prediction to notes
# and create a midi file from the notes
def create_midi(prediction_output):
    
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output.mid')

In [ ]:
create_midi(prediction_output)